In [1]:
from IPython.display import HTML, display
import csbgnpy.pd.io.sbgnml
import pypint
from tabulate import tabulate
import pintsim

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

## Phases and markers

From [2 - Markers](2%20-%20Markers.ipynb)

In [3]:
%store -r cycle_markers
%store -r cycle_phases

To ids:

In [4]:
markers = {phase: [merge.get_entity(e, by_string=True).id for e in ms]
               for phase, ms in cycle_markers.items()}

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [5]:
%store -r inis

To ids:

In [6]:
inis = {code: [e.id for e in ini] for code, ini in inis.items()}

## Dynamical models

In [7]:
ans = [pypint.load(merge_file, initial_state=ini) for ini in inis.values()]

## A. Reach markers

In [8]:
results = []
for i, (code, ini) in enumerate(sorted(inis.items())):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phase in markers:
        res = []
        for marker in markers[phase]:
            res.append(an.reachability((marker, 1), fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 1111
Computing for precursor state 1112
Computing for precursor state 1121
Computing for precursor state 1122
Computing for precursor state 1211
Computing for precursor state 1212
Computing for precursor state 1221
Computing for precursor state 1222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2121
Computing for precursor state 2122
Computing for precursor state 2211
Computing for precursor state 2212
Computing for precursor state 2221
Computing for precursor state 2222
Computing for precursor state 3111
Computing for precursor state 3112
Computing for precursor state 3121
Computing for precursor state 3122
Computing for precursor state 3211
Computing for precursor state 3212
Computing for precursor state 3221
Computing for precursor state 3222


In [9]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

Precursor state,earlyG1,lateG1,earlyS,lateS,G2,M
1111,5/5,2/2,1/1,1/1,4/4,4/4
1112,5/5,2/2,1/1,1/1,4/4,4/4
1121,5/5,2/2,1/1,1/1,4/4,4/4
1122,5/5,2/2,1/1,1/1,4/4,4/4
1211,5/5,2/2,1/1,1/1,4/4,4/4
1212,5/5,2/2,1/1,1/1,4/4,4/4
1221,5/5,2/2,1/1,1/1,4/4,4/4
1222,5/5,2/2,1/1,1/1,4/4,4/4
2111,5/5,2/2,1/1,1/1,4/4,4/4
2112,5/5,2/2,1/1,1/1,4/4,4/4


## B. Reach phases

In [10]:
results = []
for i, (code, ini) in enumerate(sorted(inis.items())):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phase in markers:
        res = []
        for marker in markers[phase]:
            l = [(marker, 1)]
            for other_phase in markers:
                if other_phase != phase:
                    l += [(marker2, 0) for marker2 in markers[other_phase]]
            if phase != "M":
                res.append(an.reachability(dict(l), reduce_for_goal=True, fallback="mole"))
            elif marker != "epn_134" or not code.startswith("2"):
                # We saw in analysis A. that marker "epn_134" of phase M is not reachable for precursor states starting with "2".
                # Reachability of all markers of phase M for the other precursor states,
                # and of all markers of phase M but "epn_134" for precursor states starting with "2" can
                # only be tested using simulation. They are all reachable.
                # Traces for these reachability properties can be found in file "CYCLE_traces_M.txt"
                reached, trace = pintsim.reachability(an, ini, dict(l), n_workers=32, max_length=1000, max_repeat=100000)
                res.append(reached)
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 1111
Computing for precursor state 1112
Computing for precursor state 1121
Computing for precursor state 1122
Computing for precursor state 1211
Computing for precursor state 1212
Computing for precursor state 1221
Computing for precursor state 1222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2121
Computing for precursor state 2122
Computing for precursor state 2211
Computing for precursor state 2212
Computing for precursor state 2221
Computing for precursor state 2222
Computing for precursor state 3111
Computing for precursor state 3112
Computing for precursor state 3121
Computing for precursor state 3122
Computing for precursor state 3211
Computing for precursor state 3212
Computing for precursor state 3221
Computing for precursor state 3222


In [11]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

Precursor state,earlyG1,lateG1,earlyS,lateS,G2,M
1111,5/5,2/2,1/1,1/1,4/4,4/4
1112,5/5,2/2,1/1,1/1,4/4,4/4
1121,5/5,2/2,1/1,1/1,4/4,4/4
1122,5/5,2/2,1/1,1/1,4/4,4/4
1211,5/5,2/2,1/1,1/1,4/4,4/4
1212,5/5,2/2,1/1,1/1,4/4,4/4
1221,5/5,2/2,1/1,1/1,4/4,4/4
1222,5/5,2/2,1/1,1/1,4/4,4/4
2111,5/5,2/2,1/1,1/1,4/4,3/3
2112,5/5,2/2,1/1,1/1,4/4,3/3


## C. Reach phases with previous phase disabled

In [12]:
results = []
couples = [(["earlyG1"], "lateG1"), (["lateG1"], "earlyS"), (["earlyS"], "lateS"), (["lateS"], "G2"), (["G2"], "M"), (["M"], "earlyG1")]
for i, (code, ini) in enumerate(sorted(inis.items())):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phases_disabled, phase in couples:
        res = []
        an_disabled = an.disable([(marker, 1) for phase_disabled in phases_disabled for marker in markers[phase_disabled]])
        for marker in markers[phase]:
            l = [(marker, 1)]
            for other_phase in markers:
                if other_phase != phase:
                    l += [(marker2, 0) for marker2 in markers[other_phase]]
            res.append(an_disabled.reachability(dict(l), fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))    
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 1111
Computing for precursor state 1112
Computing for precursor state 1121
Computing for precursor state 1122
Computing for precursor state 1211
Computing for precursor state 1212
Computing for precursor state 1221
Computing for precursor state 1222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2121
Computing for precursor state 2122
Computing for precursor state 2211
Computing for precursor state 2212
Computing for precursor state 2221
Computing for precursor state 2222
Computing for precursor state 3111
Computing for precursor state 3112
Computing for precursor state 3121
Computing for precursor state 3122
Computing for precursor state 3211
Computing for precursor state 3212
Computing for precursor state 3221
Computing for precursor state 3222


In [13]:
display(HTML(tabulate(results, headers=["Precursor state"] + ["{} -> {}".format(", ".join(phases_disabled), phase) for phases_disabled, phase in couples], tablefmt="html")))

Precursor state,earlyG1 -> lateG1,lateG1 -> earlyS,earlyS -> lateS,lateS -> G2,G2 -> M,M -> earlyG1
1111,0/2,0/1,0/1,4/4,0/4,5/5
1112,0/2,0/1,0/1,4/4,0/4,5/5
1121,0/2,0/1,0/1,4/4,0/4,5/5
1122,0/2,0/1,0/1,4/4,0/4,5/5
1211,0/2,0/1,0/1,4/4,0/4,5/5
1212,0/2,0/1,0/1,4/4,0/4,5/5
1221,0/2,0/1,0/1,4/4,0/4,5/5
1222,0/2,0/1,0/1,4/4,0/4,5/5
2111,0/2,0/1,0/1,4/4,0/4,5/5
2112,0/2,0/1,0/1,4/4,0/4,5/5
